Define a tiny example (4 frames) with reduced distributions:
- dirichlet over operations: $[1,1,1,2,1]$
- hmm: 3 states, 2 units 
- gmm: 1 1d component



In [60]:
import numpy as np
import amdtk
import subprocess
import scipy
from scipy.stats import norm 


alphas = np.array([1,1,1,2,1])
# we need n_bottom_plus * n_hmm_states * n_components gaussians
# so that's 2*3*1 = 6 gaussians 
# 
mus = np.array([-3,-2,-1,1,2,3])
# let's make all variances same
sigmas = np.array([1,1,1,1,1,1])

t = np.array([[.5, .5, 0,0,0,0],
 [0,.5,.5,0,0,0,0],
 [0,0,.5,.5,0,0,0],
 [0,0,0,.5,.5,0,0],
 [.5,0,0,0,0,0,.5]])

data = np.array([-3,-1,2,3])
tops = np.array([0,1])

# state_llh is (n_frames, n_states * n_plus)
# so that's (4, 6) 
state_llh = np.log(np.array([[norm.pdf(x, mus[i], sigmas[i]) for i in range(6)] for x in data]))
print(state_llh)

[[ -0.91893853  -1.41893853  -2.91893853  -8.91893853 -13.41893853
  -18.91893853]
 [ -2.91893853  -1.41893853  -0.91893853  -2.91893853  -5.41893853
   -8.91893853]
 [-13.41893853  -8.91893853  -5.41893853  -1.41893853  -0.91893853
   -1.41893853]
 [-18.91893853 -13.41893853  -8.91893853  -2.91893853  -1.41893853
   -0.91893853]]


In [62]:

model = amdtk.PhoneLoopNoisyChannel.create(
    n_units=2,  # number of acoustic units
    n_states=3,   # number of states per unit
    n_comp_per_state=1,   # number of Gaussians per emission
    n_top_units=2, # size of top PLU alphabet
    mean=np.zeros_like(mus), 
    var=np.ones_like(sigmas),
    max_slip_factor=.05
    #concentration=conc
)

log_prob_it = model.op_latent_posteriors[tops[0]].grad_log_partition[0]
log_prob_ib = model.op_latent_posteriors[tops[0]].grad_log_partition[1:model.n_units+1]
log_prob_sub = model.op_latent_posteriors[tops[0]].grad_log_partition[model.n_units+1:]
print("prob_it", np.exp(log_prob_it))
print("prob_ib", np.exp(log_prob_ib))
print("prob_sub", np.exp(log_prob_sub))



# ((frame_index, hmm_state, plu_bottom_type, plu_bottom_index, edit_op, plu_top_index), p)
starts = model.generate_start_items(tops, state_llh)
for tup, prob in starts:
    print(tup, np.exp(prob))





    

[-2.08333333 -2.08333333 -2.08333333 -2.08333333 -2.08333333]
[-2.08333333 -2.08333333 -2.08333333 -2.08333333 -2.08333333]
prob_it 0.124514471444
prob_ib [ 0.12451447  0.12451447]
prob_sub [ 0.12451447  0.12451447]
(0, 0, 0, 0, 0, -1) 0.0496740871809
(0, 0, 1, 0, 0, -1) 1.66637998244e-05
(-1, 2, 0, -1, 1, 0) 0.124514471444
(0, 0, 0, 0, 2, 0) 0.0496740871809
(0, 0, 1, 0, 2, 0) 1.66637998244e-05


In [66]:
# next_states(self, current_state, plu_tops, state_llh, max_slip, frames_per_top, log05, logging):
for i, state in enumerate(starts):
    nexts = model.next_states(state, tops, state_llh, .05, 2, np.log(.5), False)
    print(nexts)
    print("for state", i)

[((1, 0, 0, 0, 3, -1), -6.6143575803026238), ((1, 1, 0, 0, 3, -1), -5.1143575803026238)]
for state 0
[((1, 0, 1, 0, 3, -1), -14.614357580302624), ((1, 1, 1, 0, 3, -1), -17.11435758030262)]
for state 1
[((0, 0, 0, 0, 0, 0), -5.7787523804312837), ((0, 0, 1, 0, 0, 0), -13.778752380431284), ((0, 0, 0, 0, 2, 1), -5.7787523804312837), ((0, 0, 1, 0, 2, 1), -13.778752380431284)]
for state 2
[((1, 0, 0, 0, 3, 0), -6.6143575803026238), ((1, 1, 0, 0, 3, 0), -5.1143575803026238)]
for state 3
[((1, 0, 1, 0, 3, 0), -14.614357580302624), ((1, 1, 1, 0, 3, 0), -17.11435758030262)]
for state 4
